# Developing Model Baselines + Feature Engineering

## Setup and Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, f1_score
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import time, os, sys, warnings
import wandb
from datetime import datetime

In [26]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 10000):
        display(df)

In [3]:
from src.config import PROJECT_PATH, DATA_PATH, USE_WANDB, WANDB_PROJECT, WANDB_ENTITY
from src.tracking import ExperimentTracker

In [4]:
# Initialize the experiment tracker
tracker = ExperimentTracker(
    project_path=PROJECT_PATH,
    use_wandb=USE_WANDB,
    wandb_project_name=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY
)

In [5]:
# Load training data
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_train_demos = pd.read_csv(os.path.join(DATA_PATH, 'train_demographics.csv'))

In [6]:
# --- Create Helper Mappings for Evaluation Metric ---
# Important for the custom F1 score function
metadata = df_train[['gesture', 'sequence_type']].drop_duplicates()

# Map gesture string to sequence type (Target vs. Non-Target)
gesture_to_seq_type_map = metadata.set_index('gesture')['sequence_type'].to_dict()

# Map gesture string to integer code and back
gesture_map = {label: i for i, label in enumerate(metadata['gesture'].unique())}
inv_gesture_map = {i: label for label, i in gesture_map.items()}

# Validate
print(f"Gesture Map: {gesture_map}")
print(f"\nInverted Gesture Map: {inv_gesture_map}")
print(f"\nGesture To Sequence Type: {gesture_to_seq_type_map}")


Gesture Map: {'Cheek - pinch skin': 0, 'Forehead - pull hairline': 1, 'Write name on leg': 2, 'Feel around in tray and pull out an object': 3, 'Neck - scratch': 4, 'Neck - pinch skin': 5, 'Eyelash - pull hair': 6, 'Eyebrow - pull hair': 7, 'Forehead - scratch': 8, 'Above ear - pull hair': 9, 'Wave hello': 10, 'Write name in air': 11, 'Text on phone': 12, 'Pull air toward your face': 13, 'Pinch knee/leg skin': 14, 'Scratch knee/leg skin': 15, 'Drink from bottle/cup': 16, 'Glasses on/off': 17}

Inverted Gesture Map: {0: 'Cheek - pinch skin', 1: 'Forehead - pull hairline', 2: 'Write name on leg', 3: 'Feel around in tray and pull out an object', 4: 'Neck - scratch', 5: 'Neck - pinch skin', 6: 'Eyelash - pull hair', 7: 'Eyebrow - pull hair', 8: 'Forehead - scratch', 9: 'Above ear - pull hair', 10: 'Wave hello', 11: 'Write name in air', 12: 'Text on phone', 13: 'Pull air toward your face', 14: 'Pinch knee/leg skin', 15: 'Scratch knee/leg skin', 16: 'Drink from bottle/cup', 17: 'Glasses on/of

In [7]:
FEATURE_WAVE = "Wave 0 - Revised" # Will be updated for each wave run
N_SPLITS = 5
SEED = 42

MODEL_PARAMS = {
    'catboost': {
        'objective': 'MultiClass', # CatBoost specific
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'iterations': 1000,
        'learning_rate': 0.05,
        'depth': 6,
        'l2_leaf_reg': 3.0,
        'random_seed': SEED,
        'verbose': False,
        'allow_writing_files': False,
        'task_type' : 'GPU',
    },
    'light_gbm': {
        'objective': 'multiclass',
        'num_class': 18, # Hard coding classes
        'metric': 'multi_logloss',
        'n_estimators': 1000,
        'learning_rate': 0.05,
        'num_leaves': 31,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'bagging_fraction': 0.8,
        'feature_fraction': 0.8,
        'bagging_freq': 1,
        'random_state': SEED,
        'n_jobs': -1,
        'verbose': -1,
    }
}

In [8]:
# Revised Competition Metric for Primary Evaluation
def average_f1_score(y_true_encoded, y_pred_proba):
    """
    Calculates the competition F1 score using global maps.
    Assumes inv_gesture_map and gesture_to_seq_type_map are globally defined.
    """
    # Check for globals
    if 'inv_gesture_map' not in globals() or 'gesture_to_seq_type_map' not in globals():
        raise ValueError("Global maps 'inv_gesture_map' and 'gesture_to_seq_type_map' are required for average_f1_score.")
    
    y_pred_encoded = np.argmax(y_pred_proba, axis=1)
    
    # Convert encoded labels back to strings
    y_true_str = pd.Series(y_true_encoded).map(inv_gesture_map)
    y_pred_str = pd.Series(y_pred_encoded).map(inv_gesture_map)
    
    # Check for and handle potential mapping errors
    if y_true_str.isnull().any() or y_pred_str.isnull().any():
        raise ValueError("Error in label decoding. Check 'inv_gesture_map'.")
    
    # Binary F1-Calculation
    y_true_binary = y_true_str.map(gesture_to_seq_type_map)
    y_pred_binary = y_pred_str.map(gesture_to_seq_type_map)
    # Debugging checks for invalid sequence types
    invalid_types = set(y_true_binary.unique()) | set(y_pred_binary.unique())
    valid_types = {'Target', 'Non-Target'}
    if not invalid_types.issubset(valid_types):
        raise ValueError(f"Invalid sequence types found: {invalid_types - valid_types}")
    
    binary_f1 = f1_score(y_true_binary, y_pred_binary, 
                         pos_label='Target', average='binary')
    # Macro F1 calculation (collapsing non targets)
    def collapse_non_target(gesture):
        return 'non_target' if gesture_to_seq_type_map.get(gesture) == 'Non-Target' else gesture
    
    y_true_collapsed = y_true_str.apply(collapse_non_target)
    y_pred_collapsed = y_pred_str.apply(collapse_non_target)
    # Debugging check for invalid gestures after collapse
    invalid_collapsed = set(y_true_collapsed.unique()) | set(y_pred_collapsed.unique())
    valid_collapsed = set(gesture_to_seq_type_map.keys()) | {'non_target'}
    if not invalid_collapsed.issubset(valid_collapsed):
        raise ValueError(f"Invalid collapsed gesture types found: {invalid_collapsed - valid_collapsed}")
    
    macro_f1 = f1_score(y_true_collapsed, y_pred_collapsed, average='macro')
    return (binary_f1 + macro_f1) / 2

## Wave 0 and Pipeline Functions

In [32]:
#display_all(df_train.head(2))

In [10]:
def create_wave0_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates Wave 0 features: simple sequence-level aggregates.
    Handles ToF -1.0 values by converting to NaN before aggregation.
    """
    print("Starting Wave 0 Feature Engineering...")
    
    # --- Proactive State Management: Validate Input ---
    required_cols = ['sequence_id', 'subject', 'gesture', 'acc_x', 'acc_y', 'acc_z', 
                     'rot_w', 'rot_x', 'rot_y', 'rot_z', 'thm_1', 'thm_2', 
                     'thm_3', 'thm_4', 'thm_5']
    # Check for presence of some ToF columns (assuming they start with 'tof_')
    tof_cols_exist = any(col.startswith('tof_') for col in df.columns)
    if not tof_cols_exist:
        raise ValueError("No ToF columns (starting with 'tof_') found in DataFrame.")
        
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns for Wave 0: {missing_cols}")
    
    df_feat = df.copy()
    
    # --- Handle -1.0 in ToF columns ---
    # Identify ToF columns
    tof_columns = [col for col in df_feat.columns if col.startswith('tof_')]
    print(f"  Found {len(tof_columns)} ToF columns. Handling -1.0 values...")
    # Replace -1.0 with NaN in ToF columns before aggregation
    df_feat[tof_columns] = df_feat[tof_columns].replace(-1.0, np.nan)
    
    # --- Define Aggregations ---
    imu_cols = ['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']
    thm_cols = ['thm_1', 'thm_2', 'thm_3', 'thm_4'] # Excluding thm_5 due to high nulls
    # Note: tof_columns list is already defined above
    
    aggs = {}
    for col in imu_cols + thm_cols + tof_columns:
        aggs[col] = ['mean', 'std', 'min', 'max', 'median', 'skew']
        
    # --- Perform Aggregation ---
    print("  Performing sequence-level aggregation...")
    agg_df = df_feat.groupby('sequence_id').agg(aggs)
    
    # --- Flatten Column Names ---
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns]
    agg_df.reset_index(inplace=True)
    
    # --- Merge with Metadata ---
    print("  Merging with metadata...")
    meta_df = df_feat.groupby('sequence_id')[['subject', 'gesture']].first().reset_index(drop=True)
    # Ensure sequence_id alignment
    final_df = pd.concat([agg_df, meta_df[['subject', 'gesture']]], axis=1)
    # Defragmenting
    #final_df = pre_final_df.copy()
    #del(pre_final_df)
    
    # --- Encode Target ---
    print("  Encoding target variable...")
    # Using .astype('category').cat.codes for robustness if gesture_map not available
    # Or use gesture_map if preferred and available
    final_df['gesture_encoded'] = final_df['gesture'].astype('category').cat.codes 
    # If using gesture_map: final_df['gesture_encoded'] = final_df['gesture'].map(gesture_map)
    
    print(f"Feature engineering complete. Shape of features: {final_df.shape}")
    return final_df

In [11]:
# Create feature wave 0 
features_df_wave0 = create_wave0_features(df_train)

Starting Wave 0 Feature Engineering...
  Found 320 ToF columns. Handling -1.0 values...
  Performing sequence-level aggregation...
  Merging with metadata...
  Encoding target variable...
Feature engineering complete. Shape of features: (8151, 1990)


/tmp/ipykernel_5788/4255769560.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  agg_df.reset_index(inplace=True)


In [12]:
display_all(features_df_wave0.head())

,sequence_id,acc_x_mean,acc_x_std,acc_x_min,acc_x_max,acc_x_median,acc_x_skew,acc_y_mean,acc_y_std,acc_y_min,acc_y_max,acc_y_median,acc_y_skew,acc_z_mean,acc_z_std,acc_z_min,acc_z_max,acc_z_median,acc_z_skew,rot_w_mean,rot_w_std,rot_w_min,rot_w_max,rot_w_median,rot_w_skew,rot_x_mean,rot_x_std,rot_x_min,rot_x_max,rot_x_median,rot_x_skew,rot_y_mean,rot_y_std,rot_y_min,rot_y_max,rot_y_median,rot_y_skew,rot_z_mean,rot_z_std,rot_z_min,rot_z_max,rot_z_median,rot_z_skew,thm_1_mean,thm_1_std,thm_1_min,thm_1_max,thm_1_median,thm_1_skew,thm_2_mean,thm_2_std,thm_2_min,thm_2_max,thm_2_median,thm_2_skew,thm_3_mean,thm_3_std,thm_3_min,thm_3_max,thm_3_median,thm_3_skew,thm_4_mean,thm_4_std,thm_4_min,thm_4_max,thm_4_median,thm_4_skew,tof_1_v0_mean,tof_1_v0_std,tof_1_v0_min,tof_1_v0_max,tof_1_v0_median,tof_1_v0_skew,tof_1_v1_mean,tof_1_v1_std,tof_1_v1_min,tof_1_v1_max,tof_1_v1_median,tof_1_v1_skew,tof_1_v2_mean,tof_1_v2_std,tof_1_v2_min,tof_1_v2_max,tof_1_v2_median,tof_1_v2_skew,tof_1_v3_mean,tof_1_v3_std,tof_1_v3_min,tof_1_v3_max,tof_1_v3_median,tof_1_v3_skew,tof_1_v4_mean,tof_1_v4_std,tof_1_v4_min,tof_1_v4_max,tof_1_v4_median,tof_1_v4_skew,tof_1_v5_mean,tof_1_v5_std,tof_1_v5_min,tof_1_v5_max,tof_1_v5_median,tof_1_v5_skew,tof_1_v6_mean,tof_1_v6_std,tof_1_v6_min,tof_1_v6_max,tof_1_v6_median,tof_1_v6_skew,tof_1_v7_mean,tof_1_v7_std,tof_1_v7_min,tof_1_v7_max,tof_1_v7_median,tof_1_v7_skew,tof_1_v8_mean,tof_1_v8_std,tof_1_v8_min,tof_1_v8_max,tof_1_v8_median,tof_1_v8_skew,tof_1_v9_mean,tof_1_v9_std,tof_1_v9_min,tof_1_v9_max,tof_1_v9_median,tof_1_v9_skew,tof_1_v10_mean,tof_1_v10_std,tof_1_v10_min,tof_1_v10_max,tof_1_v10_median,tof_1_v10_skew,tof_1_v11_mean,tof_1_v11_std,tof_1_v11_min,tof_1_v11_max,tof_1_v11_median,tof_1_v11_skew,tof_1_v12_mean,tof_1_v12_std,tof_1_v12_min,tof_1_v12_max,tof_1_v12_median,tof_1_v12_skew,tof_1_v13_mean,tof_1_v13_std,tof_1_v13_min,tof_1_v13_max,tof_1_v13_median,tof_1_v13_skew,tof_1_v14_mean,tof_1_v14_std,tof_1_v14_min,tof_1_v14_max,tof_1_v14_median,tof_1_v14_skew,tof_1_v15_mean,tof_1_v15_std,tof_1_v15_min,tof_1_v15_max,tof_1_v15_median,tof_1_v15_skew,tof_1_v16_mean,tof_1_v16_std,tof_1_v16_min,tof_1_v16_max,tof_1_v16_median,tof_1_v16_skew,tof_1_v17_mean,tof_1_v17_std,tof_1_v17_min,tof_1_v17_max,tof_1_v17_median,tof_1_v17_skew,tof_1_v18_mean,tof_1_v18_std,tof_1_v18_min,tof_1_v18_max,tof_1_v18_median,tof_1_v18_skew,tof_1_v19_mean,tof_1_v19_std,tof_1_v19_min,tof_1_v19_max,tof_1_v19_median,tof_1_v19_skew,tof_1_v20_mean,tof_1_v20_std,tof_1_v20_min,tof_1_v20_max,tof_1_v20_median,tof_1_v20_skew,tof_1_v21_mean,tof_1_v21_std,tof_1_v21_min,tof_1_v21_max,tof_1_v21_median,tof_1_v21_skew,tof_1_v22_mean,tof_1_v22_std,tof_1_v22_min,tof_1_v22_max,tof_1_v22_median,tof_1_v22_skew,tof_1_v23_mean,tof_1_v23_std,tof_1_v23_min,tof_1_v23_max,tof_1_v23_median,tof_1_v23_skew,tof_1_v24_mean,tof_1_v24_std,tof_1_v24_min,tof_1_v24_max,tof_1_v24_median,tof_1_v24_skew,tof_1_v25_mean,tof_1_v25_std,tof_1_v25_min,tof_1_v25_max,tof_1_v25_median,tof_1_v25_skew,tof_1_v26_mean,tof_1_v26_std,tof_1_v26_min,tof_1_v26_max,tof_1_v26_median,tof_1_v26_skew,tof_1_v27_mean,tof_1_v27_std,tof_1_v27_min,tof_1_v27_max,tof_1_v27_median,tof_1_v27_skew,tof_1_v28_mean,tof_1_v28_std,tof_1_v28_min,tof_1_v28_max,tof_1_v28_median,tof_1_v28_skew,tof_1_v29_mean,tof_1_v29_std,tof_1_v29_min,tof_1_v29_max,tof_1_v29_median,tof_1_v29_skew,tof_1_v30_mean,tof_1_v30_std,tof_1_v30_min,tof_1_v30_max,tof_1_v30_median,tof_1_v30_skew,tof_1_v31_mean,tof_1_v31_std,tof_1_v31_min,tof_1_v31_max,tof_1_v31_median,tof_1_v31_skew,tof_1_v32_mean,tof_1_v32_std,tof_1_v32_min,tof_1_v32_max,tof_1_v32_median,tof_1_v32_skew,tof_1_v33_mean,tof_1_v33_std,tof_1_v33_min,tof_1_v33_max,tof_1_v33_median,tof_1_v33_skew,tof_1_v34_mean,tof_1_v34_std,tof_1_v34_min,tof_1_v34_max,tof_1_v34_median,tof_1_v34_skew,tof_1_v35_mean,tof_1_v35_std,tof_1_v35_min,tof_1_v35_max,tof_1_v35_median,tof_1_v35_skew,tof_1_v36_mean,tof_1_v36_std,tof_1_v36_min,tof_1_v36_max,tof_1_v36_median,tof_1_v36_skew,tof_1_v37_mean,tof_1_v37_std,tof_1_v37_

In [13]:
# --- Optional: Demonstrate sequence_counter usage (e.g., for sorting within phase if needed) ---
# This shows how sequence_counter could be used if temporal features within a phase were needed.
# It's not part of the core Wave 1 aggregations but demonstrates correct handling.
def demonstrate_sequence_counter_usage(df_sample: pd.DataFrame, head_val: int):
    """Example of using sequence_counter for sorting within groups."""
    print("\n--- Demonstrating sequence_counter usage ---")
    # Example: Sort by sequence_counter within each sequence_id and phase
    # This ensures calculations respect the order if needed later (e.g., for rolling windows within phase)
    df_sorted = df_sample.sort_values(by=['sequence_id', 'phase', 'sequence_counter'])
    print("Data sorted by sequence_id, phase, sequence_counter.")
    print(df_train.head(head_val))
    

#demonstrate_sequence_counter_usage(df_train, 100)

### Training and Evaluation

In [13]:
def train_and_evaluate_model(X: pd.DataFrame, y: pd.Series, groups: pd.Series,
                             model_type: str, model_params: dict,
                             cv_splits: int = N_SPLITS, random_seed: int = SEED):
    """
    Trains a specified model using StratifiedGroupKFold and evaluates using the custom F1 metric.
    
    Args:
        X (pd.DataFrame): Feature matrix.
        y (pd.Series): Encoded target variable.
        groups (pd.Series): Grouping variable for StratifiedGroupKFold (e.g., 'subject').
        model_type (str): Type of model ('catboost', 'xgboost', 'lightgbm').
        model_params (dict): Parameters for the model.
        cv_splits (int): Number of CV splits.
        random_seed (int): Random seed for reproducibility.
        
    Returns:
        dict: Dictionary containing model name, mean F1 score, std F1 score, and list of individual scores.
    """
    print(f"  Starting training and evaluation for {model_type.upper()}...")
    
    # State management : Validate Inputs
    if len(X) != len(y) or len(X) != len(groups):
        raise ValueError("Length of X, y and groups must be equal.")
    if model_type not in MODEL_PARAMS:
        raise ValueError(f"Unsupported model type: {model_type}. Supported types: {list(MODEL_PARAMS.keys())}")
    
    skf = StratifiedGroupKFold(n_splits=cv_splits, shuffle=True, random_state=random_seed)
    
    f1_scores = []
    models = [] # store for FI and PI
    fold_times = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y, groups)):
        print(f"    Fold {fold + 1}/{cv_splits}...")
        start_time = time.time()
        
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # --- Initialize Model ---
        if model_type == 'catboost':
            model = cat.CatBoostClassifier(**model_params)
            # CatBoost can handle categorical features if specified, but for now assuming all numeric
            model.fit(X_train_fold, y_train_fold, 
                      eval_set=(X_val_fold, y_val_fold), 
                      early_stopping_rounds=50, # Add early stopping
                      verbose=False)
        elif model_type == 'light_gbm':
            model = lgb.LGBMClassifier(**model_params)
            model.fit(X_train_fold, y_train_fold,
                      eval_set=[(X_val_fold, y_val_fold)],
                      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
        else:
            raise ValueError(f"Model type {model_type} not handled in training loop.")
        
        models.append(model) # Store the trained model
        
        # --- Predict Probabilities ---
        y_pred_proba = model.predict_proba(X_val_fold)
        
        # --- Evaluate using Custom Metric ---
        try:
            fold_f1 = average_f1_score(y_val_fold, y_pred_proba)
            f1_scores.append(fold_f1)
            print(f"      Fold {fold + 1} F1 Score: {fold_f1:.4f}")
        except Exception as e:
            print(f"      Error calculating F1 for fold {fold + 1}: {e}")
            f1_scores.append(np.nan) # Append NaN if calculation fails
            
        end_time = time.time()
        fold_times.append(end_time - start_time)
    
    # --- Calculate Final Metrics ---
    mean_f1 = np.nanmean(f1_scores) # Use nanmean in case of any NaNs
    std_f1 = np.nanstd(f1_scores)
    
    print(f"  Completed {model_type.upper()} training. Mean F1: {mean_f1:.4f} (+/- {std_f1:.4f})")
    print(f"  Average Fold Time: {np.mean(fold_times):.2f}s")
    
    return {
        'model_name': model_type.upper(),
        'mean_f1': mean_f1,
        'std_f1': std_f1,
        'fold_scores': f1_scores,
        'models': models # Return models for potential later use
    }

In [15]:
EXPERIMENT_NAME = "Wave0-Revised"

# Prepare data for training
feature_cols_wave0 = [col for col in features_df_wave0.columns if col not in ['sequence_id', 'subject', 'gesture', 'gesture_encoded']]
X_wave0 = features_df_wave0[feature_cols_wave0]
y_wave0 = features_df_wave0['gesture_encoded']
groups_wave0 = features_df_wave0['subject']

In [16]:
# Train models on Wave 0 features and save results to dict for comparison
results_wave0 = {}
for model_key in ['catboost', 'light_gbm']:
    print(f"\n--- Training {model_key.upper()} on {FEATURE_WAVE} ---")
    result = train_and_evaluate_model(
        X=X_wave0, y=y_wave0, 
        groups=groups_wave0,
        model_type=model_key, 
        model_params=MODEL_PARAMS[model_key],
        cv_splits=N_SPLITS, 
        random_seed=SEED
    )
    results_wave0[model_key] = result
    print(f"{result['model_name']} on {FEATURE_WAVE}: {result['mean_f1']:.4f} (+/- {result['std_f1']:.4f})")
    
    tracker.log_experiment(
    experiment_name=EXPERIMENT_NAME, model_name=model_key.upper(), feature_wave=FEATURE_WAVE,
    cv_score=result['mean_f1'], params=MODEL_PARAMS[model_key],
    notes="Testing revised experimentation loop"
    )



--- Training CATBOOST on Wave 0 - Revised ---
  Starting training and evaluation for CATBOOST...
    Fold 1/5...
      Fold 1 F1 Score: 0.7981
    Fold 2/5...
      Fold 2 F1 Score: 0.7231
    Fold 3/5...
      Fold 3 F1 Score: 0.7409
    Fold 4/5...
      Fold 4 F1 Score: 0.6852
    Fold 5/5...
      Fold 5 F1 Score: 0.7341
  Completed CATBOOST training. Mean F1: 0.7363 (+/- 0.0364)
  Average Fold Time: 90.55s
CATBOOST on Wave 0 - Revised: 0.7363 (+/- 0.0364)
Experiment 'Wave0-Revised' logged to ~/code/kaggle/kaggle-cmi-detect-behavior/experiment_log_revised.csv


wandb: Currently logged in as: b-a-chaudhry (b-a-chaudhry-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


cv_score,▁
cv_score,0.73628
feature_wave,Wave 0 - Revised
model_name,CATBOOST


Experiment 'Wave0-Revised' logged to W&B

--- Training LIGHT_GBM on Wave 0 - Revised ---
  Starting training and evaluation for LIGHT_GBM...
    Fold 1/5...
      Fold 1 F1 Score: 0.7848
    Fold 2/5...
      Fold 2 F1 Score: 0.7091
    Fold 3/5...
      Fold 3 F1 Score: 0.7380
    Fold 4/5...
      Fold 4 F1 Score: 0.6778
    Fold 5/5...
      Fold 5 F1 Score: 0.7252
  Completed LIGHT_GBM training. Mean F1: 0.7270 (+/- 0.0352)
  Average Fold Time: 63.07s
LIGHT_GBM on Wave 0 - Revised: 0.7270 (+/- 0.0352)
Experiment 'Wave0-Revised' logged to ~/code/kaggle/kaggle-cmi-detect-behavior/experiment_log_revised.csv


cv_score,▁
cv_score,0.72697
feature_wave,Wave 0 - Revised
model_name,LIGHT_GBM


Experiment 'Wave0-Revised' logged to W&B


## WAVE 1

In [24]:
def create_wave1_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates Wave 1 features: phase-specific statistical aggregates.
    Correctly uses 'sequence_counter' as an ordering index for calculations if needed.
    Handles ToF -1.0 values by converting to NaN before aggregation.
    """
    print("Starting Wave 1 Feature Engineering...")
    # State Management: Validate Inputs
    required_cols = ['sequence_id', 'phase', 'subject', 'gesture', 'acc_x', 'acc_y', 'acc_z', 
                     'rot_w', 'rot_x', 'rot_y', 'rot_z', 'thm_1', 'thm_2', 
                     'thm_3', 'thm_4', 'thm_5', 'sequence_counter'] # sequence_counter is key for Wave 1
    tof_cols_exist = any(col.startswith('tof_') for col in df.columns)
    if not tof_cols_exist:
        raise ValueError("No ToF columns (starting with 'tof_') found in DataFrame.")
        
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns for Wave 1: {missing_cols}")
    
    df_feat = df.copy()
    
    # Handle -1.0 in ToF columns
    tof_columns = [col for col in df_feat.columns if col.startswith('tof_')]
    print(f"  Found {len(tof_columns)} ToF columns. Handling -1.0 values...")
    df_feat[tof_columns] = df_feat[tof_columns].replace(-1.0, np.nan)
    
    # --- Define Aggregations ---
    imu_cols = ['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']
    thm_cols = ['thm_1', 'thm_2', 'thm_3', 'thm_4'] # Excluding thm_5
    # Note: tof_columns list is already defined above
    
    aggs = {}
    for col in imu_cols + thm_cols + tof_columns:
        aggs[col] = ['mean', 'std', 'min', 'max', 'median', 'skew']
        
    # ----- Addition to Wave 0 ------ Perform phase specifc aggregation
    phase_agg_df = df_feat.groupby(['sequence_id', 'phase']).agg(aggs)
    
    # Flatten column names
    phase_agg_df.columns = ['_'.join(col).strip() for col in phase_agg_df.columns]
    phase_agg_df.reset_index(inplace=True)
    
    # Unstack phase dimension
    phase_agg_df.set_index(['sequence_id', 'phase'], inplace=True)
    phase_agg_df_unstacked = phase_agg_df.unstack(level='phase')
    
    # Flatten multi-index columns and create corresponding column names
    new_cols = [f"{sensor_stat}_{phase}" for sensor_stat, phase in phase_agg_df_unstacked.columns]
    phase_agg_df_unstacked.columns = new_cols
    phase_agg_df_unstacked.reset_index(inplace=True)
    
    # Merge with Metadata
    print("  Merging with metadata...")
    meta_df = df_feat.groupby('sequence_id')[['subject', 'gesture']].first().reset_index(drop=True)
    # Ensure sequence_id alignment (phase_agg_df_unstacked should have sequence_id as a column now)
    final_df = pd.concat([phase_agg_df_unstacked, meta_df[['subject', 'gesture']]], axis=1)
    
    # --- Encode Target ---
    print("  Encoding target variable...")
    final_df['gesture_encoded'] = final_df['gesture'].astype('category').cat.codes
    # If using gesture_map: final_df['gesture_encoded'] = final_df['gesture'].map(gesture_map)
    
    print(f"Feature engineering complete. Shape of features: {final_df.shape}")
    return final_df

In [25]:
features_df_wave1 = create_wave1_features(df_train)
features_df_wave1.shape

Starting Wave 1 Feature Engineering...
  Found 320 ToF columns. Handling -1.0 values...


/tmp/ipykernel_5788/2086910628.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phase_agg_df.reset_index(inplace=True)
/tmp/ipykernel_5788/2086910628.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  phase_agg_df.reset_index(inplace=True)
/tmp/ipykernel_5788/2086910628.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use

  Merging with metadata...
  Encoding target variable...
Feature engineering complete. Shape of features: (8151, 3976)


(8151, 3976)

In [27]:
display_all(features_df_wave1.head(3))

sequence_id  acc_x_mean_Gesture  acc_x_mean_Transition  acc_x_std_Gesture  \
0  SEQ_000007            6.875854               5.227969           0.779424   
1  SEQ_000008            3.449929               3.353906           0.736728   
2  SEQ_000013           -7.591276              -6.364640           1.143683   

   acc_x_std_Transition  acc_x_min_Gesture  acc_x_min_Transition  \
0              1.334370           5.492188              3.613281   
1              1.346681           1.734375              1.886719   
2              1.162073          -9.250000             -7.714844   

   acc_x_max_Gesture  acc_x_max_Transition  acc_x_median_Gesture  \
0           9.015625              7.292969              6.964844   
1           4.949219              5.906250              3.457031   
2          -5.226562             -3.347656             -8.007812   

   acc_x_median_Transition  acc_x_skew_Gesture  acc_x_skew_Transition  \
0                 5.070312            0.671795               0.119323   
1                 2.691406           -0.292417               0.278393   
2                -6.582031            0.395099               1.138648   

   acc_y_mean_Gesture  acc_y_mean_Transition  acc_y_std_Gesture  \
0            5.640137               1.708125           0.582177   
1            7.261364               3.472433           0.824113   
2            3.751562               0.513077           0.571292   

   acc_y_std_Transition  acc_y_min_Gesture  acc_y_min_Transition  \
0              3.493182           4.683594             -2.019531   
1              3.638059           4.718750             -0.222656   
2              2.984959           2.656250             -3.273438   

   acc_y_max_Gesture  acc_y_max_Transition  acc_y_median_Gesture  \
0           6.519531              6.214844              5.621094   
1           8.667969              7.976562              7.398438   
2           4.683594              3.957031              3.746094   

   acc_y_median_Transition  acc_y_skew_Gesture  acc_y_skew_Transition  \
0                 0.277344           -0.177613               0.188431   
1                 0.546875           -0.761132               0.295310   
2                 1.695312            0.011108              -0.231933   

   acc_z_mean_Gesture  acc_z_mean_Transition  acc_z_std_Gesture  \
0            4.259521               7.265156           1.247767   
1            5.647609               7.462277           1.740919   
2           -5.493620              -6.818444           0.968642   

   acc_z_std_Transition  acc_z_min_Gesture  acc_z_min_Transition  \
0              2.335664           1.093750              3.125000   
1              2.752709           1.722656              3.175781   
2              1.382213          -6.808594            -10.945312   

   acc_z_max_Gesture  acc_z_max_Transition  acc_z_median_Gesture  \
0           6.875000              9.792969              3.910156   
1           9.000000             11.074219              5.707031   
2          -3.515625             -5.277344             -5.486328   

   acc_z_median_Transition  acc_z_skew_Gesture  acc_z_skew_Transition  \
0                 8.871094            0.123765              -0.430736   
1                 9.570312           -0.225780              -0.278592   
2                -6.566406            0.279689              -1.467629   

   rot_w_mean_Gesture  rot_w_mean_Transition  rot_w_std_Gesture  \
0            0.230537               0.305862           0.043031   
1            0.229918               0.256294           0.026109   
2            0.474384               0.285024           0.044857   

   rot_w_std_Transition  rot_w_min_Gesture  rot_w_min_Transition  \
0              0.073551           0.170898              0.134399   
1              0.084777           0.185181              0.157593   
2              0.172503           0.404968              0.061157   

   rot_w_max_Gesture  rot_w_max_Transition  rot_w_median_Gesture  \
0           0.327942

In [31]:
FEATURE_WAVE = "Wave1-Revised"

feature_cols_wave1 = [col for col in features_df_wave1.columns if col not in ['sequence_id', 'subject', 'gesture', 'gesture_encoded']]
X_wave1 = features_df_wave1[feature_cols_wave1]
y_wave1 = features_df_wave1['gesture_encoded']
groups_wave1 = features_df_wave1['subject']

In [32]:
# Train models on wave 1 features
results_wave1 = {}
for model_key in ['catboost', 'light_gbm']:
    print(f"\n---- Training {model_key.upper()} on {FEATURE_WAVE} ----")
    result = train_and_evaluate_model(
        X=X_wave1, y=y_wave1, groups=groups_wave1,
        model_type=model_key, model_params=MODEL_PARAMS[model_key],
        cv_splits=N_SPLITS, random_seed=SEED
    )
    results_wave1[model_key] = result
    print(f"{result['model_name']} on {FEATURE_WAVE}: {result['mean_f1']:.4f} (+/- {result['std_f1']:.4f})")


---- Training CATBOOST on Wave1-Revised ----
  Starting training and evaluation for CATBOOST...
    Fold 1/5...
      Fold 1 F1 Score: 0.8317
    Fold 2/5...
      Fold 2 F1 Score: 0.7837
    Fold 3/5...
      Fold 3 F1 Score: 0.7674
    Fold 4/5...
      Fold 4 F1 Score: 0.7287
    Fold 5/5...
      Fold 5 F1 Score: 0.7487
  Completed CATBOOST training. Mean F1: 0.7720 (+/- 0.0350)
  Average Fold Time: 176.36s
CATBOOST on Wave1-Revised: 0.7720 (+/- 0.0350)

---- Training LIGHT_GBM on Wave1-Revised ----
  Starting training and evaluation for LIGHT_GBM...
    Fold 1/5...
      Fold 1 F1 Score: 0.8281
    Fold 2/5...
      Fold 2 F1 Score: 0.7794
    Fold 3/5...
      Fold 3 F1 Score: 0.7655
    Fold 4/5...
      Fold 4 F1 Score: 0.7101
    Fold 5/5...
      Fold 5 F1 Score: 0.7566
  Completed LIGHT_GBM training. Mean F1: 0.7679 (+/- 0.0380)
  Average Fold Time: 103.91s
LIGHT_GBM on Wave1-Revised: 0.7679 (+/- 0.0380)


In [34]:
for model_key in results_wave1.keys():
    results_wave1[model_key] = result
    tracker.log_experiment(
        experiment_name=EXPERIMENT_NAME, model_name=model_key.upper(), feature_wave=FEATURE_WAVE,
        cv_score=result['mean_f1'], params=MODEL_PARAMS[model_key],
        notes="Testing revised experimentation loop"
        )

Experiment 'Wave0-Revised' logged to ~/code/kaggle/kaggle-cmi-detect-behavior/experiment_log_revised.csv


cv_score,▁
cv_score,0.76792
feature_wave,Wave1-Revised
model_name,CATBOOST


Experiment 'Wave0-Revised' logged to W&B
Experiment 'Wave0-Revised' logged to ~/code/kaggle/kaggle-cmi-detect-behavior/experiment_log_revised.csv


cv_score,▁
cv_score,0.76792
feature_wave,Wave1-Revised
model_name,LIGHT_GBM


Experiment 'Wave0-Revised' logged to W&B


In [ ]:
# Comparing results
comparison_data = []
for model_key in results_wave0.keys():
    comp_entry = {
        'Model': model_key.upper()
    }

{'catboost': {'model_name': 'CATBOOST',
  'mean_f1': np.float64(0.7362808034969593),
  'std_f1': np.float64(0.03641110825988725),
  'fold_scores': [0.7980651523276562,
   0.7231118870320589,
   0.7409246928436412,
   0.685173862160931,
   0.7341284231205099],
  'models': [<catboost.core.CatBoostClassifier at 0x7f6aa00a0290>,
   <catboost.core.CatBoostClassifier at 0x7f6aa449ff80>]},
 'light_gbm': {'model_name': 'LIGHT_GBM',
  'mean_f1': np.float64(0.7269747833822782),
  'std_f1': np.float64(0.035243956013649054),
  'fold_scores': [0.7848246172508884,
   0.7091028087156858,
   0.7379725989911534,
   0.6777755588287624,
   0.7251983331249006],
  'models': [LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, feature_fraction=0.8,
                  lambda_l1=0.1, lambda_l2=0.1, learning_rate=0.05,
                  metric='multi_logloss', n_estimators=1000, n_jobs=-1,
                  num_class=18, objective='multiclass', random_state=42,
                  verbose=-1),
   LGBMClassifier(